In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

# 반환값이 컬럼객체다
from pyspark.sql.functions import when, udf, col, regexp_replace
from pyspark.sql.types import DoubleType,IntegerType, StringType

# 스파크 통계
import pyspark.sql.functions as F

In [2]:
sc = SparkContext( 'local' ) # spark://ip:7070 -> 분산 클러스터 이용가능 
sqlCtx = SQLContext( sc )

In [6]:
df = sqlCtx.read.csv( path   = '../data/crime_in_Seoul.csv',
                      header = True,
                      encoding = 'euc-kr',
                      inferSchema = True # shcema 자동 추정,
                    )

In [8]:
df.head()

Row(관서명='중부서', 살인 발생=2, 살인 검거=2, 강도 발생=3, 강도 검거=2, 강간 발생=105, 강간 검거=65, 절도 발생='1,395', 절도 검거='477', 폭력 발생='1,355', 폭력 검거='1,170')

## 데이터 프레임의 rdd 속성
- 데이터 프레임에 대한 **분산리스트**
- [ Row, Row, ... ]

In [14]:
df.select('관서명').rdd.take(5)

[Row(관서명='중부서'),
 Row(관서명='종로서'),
 Row(관서명='남대문서'),
 Row(관서명='서대문서'),
 Row(관서명='혜화서')]

In [22]:
df.select('관서명').rdd.map( lambda x: x['관서명'] ).collect()

['중부서',
 '종로서',
 '남대문서',
 '서대문서',
 '혜화서',
 '용산서',
 '성북서',
 '동대문서',
 '마포서',
 '영등포서',
 '성동서',
 '동작서',
 '광진서',
 '서부서',
 '강북서',
 '금천서',
 '중랑서',
 '강남서',
 '관악서',
 '강서서',
 '강동서',
 '종암서',
 '구로서',
 '서초서',
 '양천서',
 '송파서',
 '노원서',
 '방배서',
 '은평서',
 '도봉서',
 '수서서']

In [ ]:
# df['관서명'].cast() # 데이터형 변환
# df['관서명'].isin()
# df['관서명'].like() # 문자열 
# df['관서명'].rlike() # 정규식 패턴

## where 에는 컬럼객체를 주거나 문자열을 줄 수 있다

In [29]:
df['관서명'].isin( '중부서','종로서')

Column<b'(\xea\xb4\x80\xec\x84\x9c\xeb\xaa\x85 IN (\xec\xa4\x91\xeb\xb6\x80\xec\x84\x9c, \xec\xa2\x85\xeb\xa1\x9c\xec\x84\x9c))'>

In [32]:
df.where( df['관서명'].isin( '중부서','종로서') ).show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [31]:
df.where( df['관서명'].like('%북%')).show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|성북서|        2|        2|        2|        1|       86|       71|      953|      409|    1,194|    1,015|
|강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [38]:
df.where( df['관서명'].rlike('^*대문*')).show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|    2,548|    2,227|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [43]:
df.where( df['관서명'].rlike('^.대문')).show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|    2,548|    2,227|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [51]:
df.where( df['관서명'].rlike('^[남|동|강]')).show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|    2,548|    2,227|
|  동작서|        5|        5|        9|        5|      285|      139|    1,865|      661|    1,910|    1,587|
|  강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
|  강남서|        3|        3|       15|       12|      300|      225|    2,411|      984|    2,465|    2,146|
|  강서서|        7|        8|       13|       13|      262|      191|    2,096|    1,260|    3,207|    2,718|
|  강동서|        4|        3|        6|        8|      156|      1

In [56]:
df.where( df['관서명'].rlike('강[북남]')).show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|강북서|        7|        8|       14|       13|      153|      126|    1,434|      618|    2,649|    2,348|
|강남서|        3|        3|       15|       12|      300|      225|    2,411|      984|    2,465|    2,146|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [61]:
df.where( df['관서명'].rlike('^[동서남북]대문')).show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|    2,548|    2,227|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+

